In [1]:
# Cell 1: Imports
import os
from dotenv import load_dotenv
import pandas as pd
import ast

In [2]:
from App.console_app import initialize_retriever
from App.Ingestion_chain import ingestion_chain
from App.retrieval_chain import answer_question, parse_docs
from App.VectorDB import retrieve_documents

D:\Projects\Multimodal-Retrieval-Augmented-Generation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from ragas import evaluate, EvaluationDataset
from ragas.metrics import (
    LLMContextRecall,
    Faithfulness,
    FactualCorrectness,
    ContextPrecision,
    AnswerRelevancy,
    SemanticSimilarity
)


In [4]:
from langchain_openai import ChatOpenAI
load_dotenv(verbose=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [5]:
queries = [
    "How does the Transformer architecture model sequence dependencies without using recurrence or convolution?",
    "What are the main components of the encoder stack, and how do its sub-layers operate?",
    "How does the decoder use masking to preserve the autoregressive property during generation?",
    "What mathematical steps define Scaled Dot-Product Attention and why is scaling by sqrt(dk) required?",
    "What benefits does Multi-Head Attention provide compared to a single attention head?",
    "How do positional encodings represent token order, and why do sinusoidal functions enable extrapolation?",
    "What role does the position-wise Feed-Forward Network play in each Transformer layer?",
    "How does self-attention achieve a shorter maximum path length compared to RNN and CNN architectures?",
    "What computational trade-offs does self-attention introduce in terms of per-layer complexity?",
    "What data, batching strategy, and vocabulary were used for training on the WMT 2014 English-German task?",
    "How does the learning-rate schedule with warmup affect the training dynamics of the Transformer?",
    "What regularization methods (dropout, label smoothing) were applied and why?",
    "What BLEU scores did the Transformer achieve on the WMT 2014 English-German translation benchmark?",
    "How does the Transformer's training cost compare to earlier models like GNMT or ConvS2S?",
    "What performance differences were observed when scaling from the base Transformer model to the big model?"
]

In [6]:
golden_answers = [
    "In this work we propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output. The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs.",
    "Encoder: The encoder is composed of a stack of N = 6 identical layers. Each layer has two sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-wise fully connected feed-forward network. We employ a residual connection around each of the two sub-layers, followed by layer normalization. That is, the output of each sub-layer is LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding layers, produce outputs of dimension dmodel = 512.",
    "We also modify the self-attention sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This masking, combined with the fact that the output embeddings are offset by one position, ensures that the predictions for position i can depend only on the known outputs at positions less than i. We implement this inside of scaled dot-product attention by masking out (setting to −∞) all values in the input of the softmax which correspond to illegal connections.",
    "We compute the dot products of the query with all keys, divide each by √dk, and apply a softmax function to obtain the weights on the values. In practice, we compute the attention function on a set of queries simultaneously, packed together into a matrix Q. The keys and values are also packed together into matrices K and V. We compute the matrix of outputs as: Attention(Q, K, V ) = softmax(QKT / √dk)V. While for small values of dk the two mechanisms perform similarly, additive attention outperforms dot product attention without scaling for larger values of dk. We suspect that for large values of dk, the dot products grow large in magnitude, pushing the softmax function into regions where it has extremely small gradients. To counteract this effect, we scale the dot products by √1/dk.",
    "Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this.",
    "In this work, we use sine and cosine functions of different frequencies: PE(pos,2i) = sin(pos/10000^(2i/dmodel)) and PE(pos,2i+1) = cos(pos/10000^(2i/dmodel)), where pos is the position and i is the dimension. We chose this function because we hypothesized it would allow the model to easily learn to attend by relative positions, since for any fixed offset k, PEpos+k can be represented as a linear function of PEpos. We also experimented with using learned positional embeddings instead, and found that the two versions produced nearly identical results. We chose the sinusoidal version because it may allow the model to extrapolate to sequence lengths longer than the ones encountered during training.",
    "In addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully connected feed-forward network, which is applied to each position separately and identically. This consists of two linear transformations with a ReLU activation in between. While the linear transformations are the same across different positions, they use different parameters from layer to layer.",
    "As noted in Table 1, a self-attention layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence length n is smaller than the representation dimensionality d. A single convolutional layer with kernel width k < n does not connect all pairs of input and output positions, and doing so requires a stack of O(n/k) convolutional layers or O(log_k(n)) in the case of dilated convolutions, increasing the length of the longest paths between any two positions in the network.",
    "One is the total computational complexity per layer. Another is the amount of computation that can be parallelized, as measured by the minimum number of sequential operations required. As noted in Table 1, the complexity per self-attention layer is O(n^2 · d). Convolutional layers are generally more expensive than recurrent layers, by a factor of k, though separable convolutions decrease the complexity considerably. Even with k = n, however, the complexity of a separable convolution is equal to the combination of a self-attention layer and a point-wise feed-forward layer, the approach we take in our model.",
    "We trained on the standard WMT 2014 English-German dataset consisting of about 4.5 million sentence pairs. Sentences were encoded using byte-pair encoding, which has a shared source-target vocabulary of about 37000 tokens. Sentence pairs were batched together by approximate sequence length. Each training batch contained a set of sentence pairs containing approximately 25000 source tokens and 25000 target tokens.",
    "We varied the learning rate over the course of training, according to the formula: lrate = d^(-0.5)_model · min(step_num^(-0.5), step_num · warmup_steps^(-1.5)). This corresponds to increasing the learning rate linearly for the first warmup_steps training steps, and decreasing it thereafter proportionally to the inverse square root of the step number.",
    "We employ three types of regularization during training: Residual Dropout. We apply dropout to the output of each sub-layer, before it is added to the sub-layer input and normalized. In addition, we apply dropout to the sums of the embeddings and the positional encodings in both the encoder and decoder stacks. For the base model, we use a rate of P_drop = 0.1. Label Smoothing. During training, we employed label smoothing of value ε_ls = 0.1. This hurts perplexity, as the model learns to be more unsure, but improves accuracy and BLEU score.",
    "On the WMT 2014 English-to-German translation task, the big transformer model outperforms the best previously reported models (including ensembles) by more than 2.0 BLEU, establishing a new state-of-the-art BLEU score of 28.4. Even our base model surpasses all previously published models and ensembles, at a fraction of the training cost of any of the competitive models. Table 2: The Transformer achieves better BLEU scores than previous state-of-the-art models on the English-to-German and English-to-French newstest2014 tests at a fraction of the training cost.",
    "Table 2 summarizes our results and compares our translation quality and training costs to other model architectures from the literature. We estimate the number of floating point operations used to train a model by multiplying the training time, the number of GPUs used, and an estimate of the sustained single-precision floating-point capacity of each GPU. The Transformer (base model) EN-DE training cost is listed as 3.3 · 10^18 FLOPs, compared to GNMT + RL at 2.3 · 10^19 FLOPs, ConvS2S at 9.6 · 10^18 FLOPs, and ByteNet, Deep-Att + PosUnk, MoE and others at significantly higher costs.",
    "In Table 3 rows (C) and (D) we observe that, as expected, bigger models are better. On the WMT 2014 English-to-German translation task, the big transformer model achieves a BLEU score of 28.4, compared to the base model's reported 27.3 in Table 2. For the big model, training took 3.5 days on 8 P100 GPUs, while the base model trained for 12 hours. The big model used d_model = 1024, d_ff = 4096, h = 16, and P_drop = 0.3, compared to the base model's smaller configuration."
]

In [7]:
ret = initialize_retriever()

# Ingest your PDF (CRITICAL: This populates both vectorstore AND docstore)
file = r"D:\Projects\Multimodal-Retrieval-Augmented-Generation\uploaded_pdfs\NIPS-2017-attention-is-all-you-need-Paper.pdf"
ingestion_chain(file, ret)

print("\n✅ Retriever initialized and PDF ingested!")
print(f"📊 Vectorstore collection: {ret.vectorstore._collection.name}")
print(f"📦 Docstore contains: {len(ret.docstore.store)} original documents")


✅ Vector database initialized at: D:\Projects\Multimodal-Retrieval-Augmented-Generation\chroma_store
✅ Ollama is running.
✅ Extracted 9 elements from PDF.
📄 Texts: 9, 📊 Tables: 0, 🖼️ Images: 3
✅ Summarization complete.
✅ Added 9 text summaries.
⚠️ No table summaries to add.
✅ Added 3 image summaries.
✅ Documents added to vector database.

✅ Retriever initialized and PDF ingested!
📊 Vectorstore collection: multi_modal_rag
📦 Docstore contains: 12 original documents


In [8]:
def build_ragas_dataset_v2(queries, gold_answers, retriever, output="ragas_testset_v2.csv"):
    """
    NEW VERSION: Retrieves ORIGINAL documents (not summaries)
    Uses the MultiVectorRetriever that returns parent documents.
    """

    rows = []

    for q, ref in zip(queries, gold_answers):
        print(f"Processing: {q[:60]}...")

        # --- RETRIEVE ORIGINAL DOCUMENTS ---
        # MultiVectorRetriever searches summaries but returns ORIGINAL content
        docs = retrieve_documents(retriever=retriever, question=q, k=4)

        # --- EXTRACT CONTEXT ---
        # Parse docs returns {"images": [...], "texts": [...]}
        context_obj = parse_docs(docs)

        # Convert to strings for RAGAS
        text_contexts = []
        for doc in context_obj["texts"]:
            if hasattr(doc, 'page_content'):
                text_contexts.append(doc.page_content)
            elif hasattr(doc, 'text'):  # For unstructured elements
                text_contexts.append(doc.text)
            else:
                text_contexts.append(str(doc))

        # Images as base64 (RAGAS can't directly evaluate these, but we keep for completeness)
        image_contexts = context_obj["images"]

        # Combine all contexts
        retrieved_contexts = text_contexts + [f"[IMAGE: base64 data of length {len(img)}]" for img in image_contexts]

        # --- GENERATE ANSWER ---
        # This uses your full retrieval chain (including images in prompt)
        answer = answer_question(q, retriever)

        # --- ADD RAGAS ROW ---
        rows.append({
            "user_input": q,
            "retrieved_contexts": retrieved_contexts,  # List of strings
            "response": answer,
            "reference": ref
        })

    # --- SAVE CSV ---
    df = pd.DataFrame(rows)
    df.to_csv(output, index=False)
    print(f"\n✅ Saved RAGAS dataset → {output}")

    return df


In [9]:
testset_df = build_ragas_dataset_v2(queries, golden_answers, ret, output="ragas_testset_v2.csv")


Processing: How does the Transformer architecture model sequence depende...
Processing: What are the main components of the encoder stack, and how d...
Processing: How does the decoder use masking to preserve the autoregress...
Processing: What mathematical steps define Scaled Dot-Product Attention ...
Processing: What benefits does Multi-Head Attention provide compared to ...
Processing: How do positional encodings represent token order, and why d...
Processing: What role does the position-wise Feed-Forward Network play i...
Processing: How does self-attention achieve a shorter maximum path lengt...
Processing: What computational trade-offs does self-attention introduce ...
Processing: What data, batching strategy, and vocabulary were used for t...
Processing: How does the learning-rate schedule with warmup affect the t...
Processing: What regularization methods (dropout, label smoothing) were ...
Processing: What BLEU scores did the Transformer achieve on the WMT 2014...
Processing: 

In [10]:
data = pd.read_csv("ragas_testset_v2.csv")

# Convert string representation of lists back to actual lists
data["retrieved_contexts"] = data["retrieved_contexts"].apply(ast.literal_eval)

# Create RAGAS EvaluationDataset
evaluation_dataset = EvaluationDataset.from_pandas(data)

print(f"\n✅ Evaluation dataset ready with {len(data)} samples")


✅ Evaluation dataset ready with 15 samples


In [11]:
evaluator_llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=OPENAI_API_KEY,
    temperature=0
)

# Define metrics
metrics = [
    LLMContextRecall(),
    ContextPrecision(),
    Faithfulness(),
    FactualCorrectness(),
    AnswerRelevancy(),
    SemanticSimilarity(),
]

print("\n🔄 Running RAGAS evaluation...")
result = evaluate(
    dataset=evaluation_dataset,
    metrics=metrics,
    llm=evaluator_llm,
)

print("\n✅ Evaluation complete!")



🔄 Running RAGAS evaluation...


Evaluating:   0%|          | 0/90 [00:00<?, ?it/s]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating:  20%|██        | 18/90 [02:05<05:42,  4.76s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating:  22%|██▏       | 20/90 [03:03<11:19,  9.70s/it]Exception raised in Job[13]: TimeoutError()
Exception raised in Job[7]: TimeoutError()
Evaluating: 100%|██████████| 90/90 [09:40<00:00,  6.45s/it]



✅ Evaluation complete!


In [12]:
results = result.to_pandas()

# Save results
results.to_csv("ragas_results_v2.csv", index=False)
print("✅ Saved metrics → ragas_results_v2.csv")

# Display summary statistics
print("\n" + "="*60)
print("📊 RAGAS EVALUATION SUMMARY")
print("="*60)

metric_cols = [
    'context_recall',
    'context_precision',
    'faithfulness',
    'factual_correctness(mode=f1)',
    'answer_relevancy',
    'semantic_similarity'
]

summary = results[metric_cols].describe().T[['mean', 'std', 'min', 'max']]
print(summary)

✅ Saved metrics → ragas_results_v2.csv

📊 RAGAS EVALUATION SUMMARY
                                  mean       std       min       max
context_recall                0.916667  0.191589  0.333333  1.000000
context_precision             0.981481  0.036747  0.916667  1.000000
faithfulness                  0.800608  0.189847  0.400000  1.000000
factual_correctness(mode=f1)  0.490714  0.215637  0.200000  0.800000
answer_relevancy              0.960015  0.035466  0.902491  1.000000
semantic_similarity           0.934785  0.021089  0.904920  0.971087
